In [1]:
import numpy as np
from skimage.io import imread
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV
from os.path import join
from numpy import zeros

**Функции, которые находят модуль градиента и его направление (сразу приводим к значению $\in [0, 180[$):**

In [2]:
def brightness(image):
    return image[:, :, 0] * 0.299 + \
           image[:, :, 1] * 0.587 + \
           image[:, :, 2] * 0.114


def derivative_x(im_bness):
    return np.concatenate((im_bness[:1, :], im_bness[:-1, :]), axis=0) - \
           np.concatenate((im_bness[1:, :], im_bness[-1:, :]), axis=0)


def derivative_y(im_bness):
    return np.concatenate((im_bness[:, :1], im_bness[:, :-1]), axis=1) - \
           np.concatenate((im_bness[:, 1:], im_bness[:, -1:]), axis=1)


def gradient_abs_and_dir(im_bness):
    der_x = derivative_x(im_bness)
    der_y = derivative_y(im_bness)
    return np.array(np.sqrt(der_x * der_x + der_y * der_y), dtype='float64'), \
           (np.arctan2(der_y, der_x) * 180 / np.pi) % 180

**Метод определения номера корзины для углов, $bin\_count = 9$:**

In [3]:
def basket_number(pos_angle, bin_count):
    result = np.zeros(pos_angle.shape, dtype=int)
    basket_size = 180 / bin_count
    left_border = 0
    for j in range(bin_count):
        right_border = left_border + basket_size
        result[np.logical_and(pos_angle >= left_border, pos_angle < right_border)] = j
        left_border += basket_size
    return result

**Функции построения гистограммы направлений для некоторой ячейки, а также для всего изображения, разбитого на ячейки. Все изображения в функции приводятся к размеру (112, 112, 3), поэтому число ячеек по строкам и столбцам равно 14.**

In [4]:
def cell_histogram(basket_num, cell_abs, bin_count):
    sums = np.zeros(bin_count)
    for i in range(bin_count):
        sums[i] += np.sum(cell_abs[basket_num == i])
    return sums


def full_histogram(basket_num, grad_abs, bin_count):
    n_cells = 14
    cell_rows = int(basket_num.shape[0] / n_cells)
    cell_cols = int(basket_num.shape[1] / n_cells)
    height = basket_num.shape[0]
    width = basket_num.shape[1]
    result = np.zeros((n_cells, n_cells, bin_count))
    u_border = 0
    for i in range(n_cells):
        d_border = u_border + cell_rows
        if i == n_cells - 1:
            d_border = height
        l_border = 0
        for j in range(n_cells):
            r_border = l_border + cell_cols
            if j == n_cells - 1:
                r_border = width
            result[i, j] = cell_histogram(basket_num[u_border:d_border, l_border:r_border],
                                                grad_abs[u_border:d_border, l_border:r_border],
                                                bin_count)
            l_border += cell_cols
        u_border += cell_rows
    return result

**Аналогично строится метод, возвращающий нормированный вектор гистограмм ячеек блока, который после применён для построения дескриптора изображения. В этом методе блоки  пересекаются пересечений столько, чтобы длина дескриптора была приемлемой в плане расходуемой памяти, но при этом дающая хороший результат.**

In [5]:
def block_descriptor(block, eps):
    sub_vectors = np.zeros((block.shape[0], block.shape[1] * block.shape[2]))
    for i in range(block.shape[0]):
        sub_vectors[i] = np.concatenate(block[i, :])
    vector = np.concatenate(sub_vectors[:])
    return vector / np.sqrt(np.sum(vector * vector) + eps)


def descriptor(full_hist, block_shape, eps):
    block_rows = block_shape[0]
    block_cols = block_shape[1]
    b_height = full_hist.shape[0]
    b_width = full_hist.shape[1]
    result = np.zeros((0,))
    for u_border in range(0, b_height-block_rows + 1):
        d_border = u_border + block_rows
        for l_border in range(0, b_width-block_cols + 1):
            r_border = l_border + block_cols
            if u_border % 5 != 0 and l_border % 5 != 0:
                result = np.concatenate((result,
                                         block_descriptor(full_hist[u_border:d_border,
                                                          l_border:r_border], eps)))
    return result

**Итоговая функция extract_hog:**

In [6]:
def extract_hog(image):
    opt_size = (112, 112, 3)
    bin_count = 8
    resized = resize(image, opt_size, mode='reflect')
    im_bness = brightness(resized)
    grad_abs, grad_dir = gradient_abs_and_dir(im_bness)
    baskets = basket_number(grad_dir, bin_count)
    histogram = full_histogram(baskets, grad_abs, bin_count)
    return descriptor(histogram, (2, 2), 1e-100)


def fit_and_classify(train_features, train_labels, test_features):
    svc = LinearSVC(dual=False, C=3)
    svc.fit(train_features, train_labels)
    return svc.predict(test_features)

**Для подбора оптимальных гиперпараметров был применён рандомизированный поиск, который дал результат: $С = 3$.**

In [7]:
def best_params_svm(train_features, train_labels):
    svc = LinearSVC()
    params = {'C': [0.5, 1., 1.5, 0.2, 0.8, 2., 3.]}
    search = RandomizedSearchCV(svc, params, n_iter=4, n_jobs=-1, cv=5, scoring='accuracy')
    search.fit(train_features, train_labels)
    return search.best_params_
